<a href="https://colab.research.google.com/github/soumyadip1995/language-models/blob/main/Multi_head_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
words = open('/content/text.txt', 'r').read().split()
words[:20]

['So',
 'Ray',
 'J',
 'went',
 'straight',
 'to',
 'the',
 'radio',
 'station',
 'The',
 'very',
 'next',
 'day,',
 '"Hey',
 'Fab,',
 "I'ma",
 'kill',
 'you!"',
 'Lyrics',
 "comin'"]

In [70]:
import torch
from torch import nn

chars = sorted(list(set(words)))
string2integer = {ch: i for i, ch in enumerate(chars)}
print(string2integer)

integer2string = {i:ch for ch,i in string2integer.items()}
encode = lambda s: [string2integer[c] for c in s]
print(encode)

decode = lambda l: ''.join([integer2string[i] for i in l])
print(decode)

data = torch.tensor(encode(words), dtype = torch.float)
print(data)

{'"': 0, '"Hey': 1, '"It\'s': 2, '"Lose': 3, '"Oh,': 4, "'Cause": 5, "'cause": 6, "'em": 7, '(J.J.': 8, 'Doc': 9, 'Fab,': 10, 'Fad)': 11, 'For': 12, 'How': 13, 'I': 14, "I'm": 15, "I'ma": 16, 'Innovative': 17, 'J': 18, 'Lyrics': 19, 'Never': 20, 'Ray': 21, 'So': 22, 'The': 23, 'Throw': 24, 'Uh,': 25, 'Well,': 26, 'What': 27, 'With': 28, 'You': 29, 'Yourself"': 30, 'a': 31, 'and': 32, 'anything': 33, 'are': 34, "assumin'": 35, 'at': 36, 'audience': 37, 'be': 38, 'can': 39, 'celebrating': 40, "comin'": 41, 'confuse': 42, 'day': 43, 'day,': 44, 'demonstrating': 45, 'devastating,': 46, 'do': 47, 'dooma-lumma,': 48, 'elevating': 49, 'elevator': 50, 'ever': 51, 'fading,': 52, 'feeling': 53, 'fell': 54, 'forever': 55, 'found': 56, 'fuse': 57, 'get': 58, 'give': 59, 'glue': 60, 'gotta': 61, 'haters': 62, "he's": 63, 'hella': 64, 'hip-hop,': 65, 'human': 66, 'is': 67, 'it': 68, "it'll": 69, "it's": 70, 'jealous,': 71, 'kill': 72, 'know': 73, 'levitating': 74, 'like': 75, 'lose': 76, 'made': 77,

In [71]:
# - blocksize, (batchsize))
ix = torch.randint(len(data) - 64, (512,))
ix

tensor([ 90,  35,  82,  79,  43,  57, 102,  69,  68,  98,  83,  68,  83,  59,
          5,  89,  61,  32,  27,  71,  24,  24,  19,  96,  33,  60,  40,  94,
         18,  38,  79,  70, 107,  82,  88,  74,   8,  58,  37,   6, 100,  48,
         59,  15,  99,  31,  56,  60,  97,   7,  86,   6,  30,  33, 107,  81,
         68, 107,  57,  94,  83,  81,  14,  79,  70,  27,  71,  65,  37,   7,
         66,  77,  78,  61,  70,  75,   6,  31,  16,  49,  78,  84,  28,  97,
        100,  72,  99,  92,  58,  59,  94,  56,  41,  10,  98,  74,  54,   0,
         60,   4, 103,  78,  43, 104,  43,  93,  26,  20,  58,  52,   5,   7,
         96, 107, 107,  84,  30,  58,  33,   4, 101,  88,  45,  23,  65,  19,
         97,  11,  73,  38,  37,  38,  61,  61,  23,  50,  68,  74,  74,  86,
         69,  84,  31,  53,  49,  64, 103,  94,   3,  51,  36, 104,  33,  31,
         78,  55,  72, 102,  82,  43,  70,  77,  64,  17,  32,  26,  94,  86,
         46,  65,  16,  78,   8,  78,  21,  18,   9,   0,  48,  

In [72]:
x = torch.stack([data[i:i+64] for i in ix])

d = x.float()
d


tensor([[ 73., 110.,  62.,  ...,  31.,  64., 118.],
        [ 27.,  14.,  61.,  ...,  12., 110.,  43.],
        [ 53.,  75.,  70.,  ...,  87.,  65.,  70.],
        ...,
        [110.,  62.,  34.,  ...,  64., 118., 114.],
        [ 19.,  41.,  36.,  ...,  82.,  37.,  31.],
        [108., 111.,  39.,  ...,  94., 122.,   9.]])

In [73]:

p = d.transpose(-2, 1)
p.size()

torch.Size([64, 512])

In [74]:
import math
import numpy as np 
import torch.nn.functional  as F
import torch
from math import sqrt


def scaled_dot_product_attention(query, key, value):
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(-2, -1)) / sqrt(dim_k)
  weights = F.softmax(scores, dim = -1)
  return torch.bmm(weights, value)
  


In [75]:
import torch
chars = sorted(list(set(words)))
# print(chars)
size = len(chars) # sequence length
# print(size)

torch.manual_seed(1337)
dim_k = 2


key = torch.randn(512, size, dim_k)
value = torch.randn(512, size, dim_k) 
query = torch.randn(512, size, dim_k)



sdp = scaled_dot_product_attention(query, key, value)
sdp.size()

torch.Size([512, 125, 2])

In [76]:
class AttentionHead(nn.Module):
  def __init__(self, embedded_dim, head_dim):
    super().__init__()
    self.q = nn.Linear(embedded_dim, head_dim, bias = False)
    self.k = nn.Linear(embedded_dim,  head_dim, bias = False)
    self.v = nn.Linear(embedded_dim,  head_dim, bias = False)

  def forward(self, p):
    attention_outputs = scaled_dot_product_attention(self.q(p), self.k(p), self.v(p))
    

    return attention_outputs
    

In [77]:
# embedding_dim = embedding dimensions
# num_heads  = number of heads 


class MultiHeadAttention(nn.Module):
  def __init__(self, embedded_dim, num_heads):
    super().__init__()
    self.embedded_dim = embedded_dim
    self.num_heads = num_heads
    head_dim = embedded_dim // num_heads 

    self.heads = nn.ModuleList([AttentionHead(embedded_dim, head_dim) for _ in range(num_heads)])
    self.output_linear = nn.Linear(embedded_dim, embedded_dim)

  def forward(self, p):
    out = torch.cat([h(p) for h in self.heads], dim = -1)
    
    out = self.output_linear(out)

    return out


In [83]:
multihead_attention = MultiHeadAttention(embedded_dim = 512, num_heads = 8)
multihead_attention

MultiHeadAttention(
  (heads): ModuleList(
    (0): AttentionHead(
      (q): Linear(in_features=512, out_features=64, bias=False)
      (k): Linear(in_features=512, out_features=64, bias=False)
      (v): Linear(in_features=512, out_features=64, bias=False)
    )
    (1): AttentionHead(
      (q): Linear(in_features=512, out_features=64, bias=False)
      (k): Linear(in_features=512, out_features=64, bias=False)
      (v): Linear(in_features=512, out_features=64, bias=False)
    )
    (2): AttentionHead(
      (q): Linear(in_features=512, out_features=64, bias=False)
      (k): Linear(in_features=512, out_features=64, bias=False)
      (v): Linear(in_features=512, out_features=64, bias=False)
    )
    (3): AttentionHead(
      (q): Linear(in_features=512, out_features=64, bias=False)
      (k): Linear(in_features=512, out_features=64, bias=False)
      (v): Linear(in_features=512, out_features=64, bias=False)
    )
    (4): AttentionHead(
      (q): Linear(in_features=512, out_featur

In [79]:
#### Single headed attention

import torch
import torch.nn as nn

class SingleAttention(nn.Module):
    def __init__(self, embedded_dim):
        super(SingleAttention, self).__init__()
        self.embedded_dim = embedded_dim
        
        self.W_q = nn.Linear(embedded_dim, embedded_dim)
        self.W_k = nn.Linear(embedded_dim, embedded_dim)
        self.W_v = nn.Linear(embedded_dim, embedded_dim)
        self.W_o = nn.Linear(embedded_dim, embedded_dim)
        
    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)
        
        # Apply linear transformation to Q, K, and V
        Q = self.W_q(Q)
        K = self.W_k(K)
        V = self.W_v(V)
        
        # Compute attention scores and weights
        scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.embedded_dim).float())
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        weights = nn.Softmax(dim=-1)(scores)
        
        # Apply attention weights to values
        output = torch.matmul(weights, V)
        output = self.W_o(output)
        
        return output


In [80]:



import torch
chars = sorted(list(set(words)))
# print(chars)
seq_len = len(chars) # sequence length
embedded_dim = 512
batch_size = 32

# Create some dummy input tensors
Q = torch.randn(batch_size, seq_len, embedded_dim)
K = torch.randn(batch_size, seq_len, embedded_dim)
V = torch.randn(batch_size, seq_len, embedded_dim)

# Create an instance of the single attention model
single_attn = SingleAttention(embedded_dim)

# Pass the input tensors through the model
output = single_attn(Q, K, V)

# The output tensor has shape (batch_size, seq_len, embedded_dim)
output.shape

torch.Size([32, 125, 512])